# 1. Install Dependencies and Setup

In [ ]:
# !pip install labelme tensorflow tensorflow-gpu opencv-python matplotlib albumentations

# 2. Collect Images

In [1]:
import os
import time
import uuid
import cv2
from glob import glob
from tqdm import tqdm
import numpy as np
import albumentations as alb
import json

In [2]:
main_dir = r'G:\Ayman\Future things\AI Projects\Iris Detection\iris_data'
imgs_path = r'G:\Ayman\Future things\AI Projects\Iris Detection\iris_data\images'
labels_path = r'G:\Ayman\Future things\AI Projects\Iris Detection\iris_data\labels'
number_images = 150

In [3]:
cap = cv2.VideoCapture(0)
for imgnum in range(number_images):
    print('Collecting image {}'.format(imgnum))
    ret, frame = cap.read()
    imgname = os.path.join(imgs_path,f'{str(uuid.uuid1())}.jpg')
    cv2.imwrite(imgname, frame)
    cv2.imshow('frame', frame)
    time.sleep(0.5)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

# 3. Create Dataset

In [4]:
import tensorflow as tf
import cv2
import json
import numpy as np
from matplotlib import pyplot as plt

In [5]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [6]:
def load_image(x): 
    byte_img = tf.io.read_file(x)
    img = tf.io.decode_jpeg(byte_img)
    return img

# 4. Move Matching Labels AFTER Annotation with Labelme

In [15]:
import os
import shutil

# Define the paths
main_dir = r'G:\Ayman\Future things\AI Projects\Iris Detection\iris_data'
imgs_path = r'G:\Ayman\Future things\AI Projects\Iris Detection\iris_data\images'
labels_path = r'G:\Ayman\Future things\AI Projects\Iris Detection\iris_data\labels'

train_dir = os.path.join(main_dir, 'train')
test_dir = os.path.join(main_dir, 'test')
val_dir = os.path.join(main_dir, 'val')

# Create the train, test, and val directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Create the 'images' and 'labels' folders for each subset
os.makedirs(os.path.join(train_dir, 'images'), exist_ok=True)
os.makedirs(os.path.join(train_dir, 'labels'), exist_ok=True)
os.makedirs(os.path.join(test_dir, 'images'), exist_ok=True)
os.makedirs(os.path.join(test_dir, 'labels'), exist_ok=True)
os.makedirs(os.path.join(val_dir, 'images'), exist_ok=True)
os.makedirs(os.path.join(val_dir, 'labels'), exist_ok=True)

# Get the list of image files
image_files = os.listdir(imgs_path)

# Calculate the number of files for each split
num_images = len(image_files)
num_train = int(0.8 * num_images)
num_test = int(0.1 * num_images)
num_val = num_images - num_train - num_test

# Split the image files
train_files = image_files[:num_train]
test_files = image_files[num_train:num_train+num_test]
val_files = image_files[num_train+num_test:]

# Copy the image files to the train directory
for file in train_files:
    src = os.path.join(imgs_path, file)
    dst = os.path.join(train_dir, 'images', file)
    shutil.copy(src, dst)

# Copy the image files to the test directory
for file in test_files:
    src = os.path.join(imgs_path, file)
    dst = os.path.join(test_dir, 'images', file)
    shutil.copy(src, dst)

# Copy the image files to the val directory
for file in val_files:
    src = os.path.join(imgs_path, file)
    dst = os.path.join(val_dir, 'images', file)
    shutil.copy(src, dst)

# Copy the label files to the train directory
for file in train_files:
    filename = file.split('.')[0] + '.json'
    src = os.path.join(labels_path, filename)
    dst = os.path.join(train_dir, 'labels', filename)
    shutil.copy(src, dst)

# Copy the label files to the test directory
for file in test_files:
    filename = file.split('.')[0] + '.json'
    src = os.path.join(labels_path, filename)
    dst = os.path.join(test_dir, 'labels', filename)
    shutil.copy(src, dst)

# Copy the label files to the val directory
for file in val_files:
    filename = file.split('.')[0] + '.json'
    src = os.path.join(labels_path, filename)
    dst = os.path.join(val_dir, 'labels', filename)
    shutil.copy(src, dst)

In [16]:
for folder in ['train','test','val']:
    for file in os.listdir(os.path.join(main_dir, folder, imgs_path)):
        
        filename = file.split('.')[0]+'.json'
        existing_filepath = os.path.join(main_dir,labels_path, filename)
        if os.path.exists(existing_filepath): 
            new_filepath = os.path.join(main_dir,folder,labels_path,filename)
            os.replace(existing_filepath, new_filepath)  

# 5. Image Augmentation

In [2]:
augmentor = alb.Compose([
    alb.HorizontalFlip(p=0.5),  # Flip horizontally to add variation.
    
    alb.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),  # Subtle brightness and contrast adjustments.
    
    alb.RandomGamma(gamma_limit=(80, 120), p=0.5),  # Adjust gamma to change brightness while preserving contrast.
    
    alb.GaussNoise(var_limit=(10.0, 50.0), p=0.5),  # Add moderate Gaussian noise for robustness.
    
    alb.RGBShift(r_shift_limit=10, g_shift_limit=10, b_shift_limit=10, p=0.5),  # Subtle color shifts to simulate lighting variations.
    
    alb.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=20, p=0.5),  # Small shifts, scaling, and rotations to augment without distorting keypoints.
    
    alb.Resize(height=720, width=720),  # Resize all images to a consistent size without cropping or distortion.
    
    alb.CoarseDropout(max_holes=30, max_height=16, max_width=16, min_holes=10, fill_value=0, p=0.5),  # Reduce dropout to maintain keypoints visibility.
    
    # alb.CLAHE(p=0.3)  # Optional: Use this if contrast-limited histogram equalization is beneficial.
    ],
    keypoint_params=alb.KeypointParams(format='xy', label_fields=['class_labels'])
)

In [3]:
main_dir = '/home/ayman/AI Projects/Iris Detection/iris_data/'
aug_path = '/home/ayman/AI Projects/Iris Detection/iris_data/aug_data'

In [4]:
for partition in ['train', 'test', 'val']: 
    for image in os.listdir(os.path.join(main_dir, partition, 'images')):
        img = cv2.imread(os.path.join(main_dir, partition, 'images', image))

        classes = [0,0]
        coords = [0,0,0.00001,0.00001]
        label_path = os.path.join(main_dir, partition, 'labels', f'{image.split(".")[0]}.json')
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                label = json.load(f)
    
            if label['shapes'][0]['label']=='L_eye': 
                classes[0] = 1
                coords[0] = np.squeeze(label['shapes'][0]['points'])[0]
                coords[1] = np.squeeze(label['shapes'][0]['points'])[1]

            if label['shapes'][0]['label']=='R_eye':
                classes[1] = 1
                coords[2] = np.squeeze(label['shapes'][0]['points'])[0]
                coords[3] = np.squeeze(label['shapes'][0]['points'])[1]

            if len(label['shapes']) > 1:     
                if label['shapes'][1]['label'] =='L_eye': 
                    classes[0] = 1 
                    coords[0] = np.squeeze(label['shapes'][1]['points'])[0]
                    coords[1] = np.squeeze(label['shapes'][1]['points'])[1]

                if label['shapes'][1]['label'] =='R_eye': 
                    classes[1] = 1
                    coords[2] = np.squeeze(label['shapes'][1]['points'])[0]
                    coords[3] = np.squeeze(label['shapes'][1]['points'])[1]
            
            np.divide(coords, [1280,720,1280,720])
                
        try: 
            for x in range(50):
                keypoints = [(coords[:2]), (coords[2:])]
                augmented = augmentor(image=img, keypoints=keypoints, class_labels=['L_eye','R_eye'])
                cv2.imwrite(os.path.join(aug_path, partition, 'images', f'{image.split(".")[0]}.{x}.png'), augmented['image'])

                annotation = {}
                annotation['image'] = image
                annotation['class'] = [0,0]
                annotation['keypoints'] = [0,0,0,0]

                if os.path.exists(label_path):
                    if len(augmented['keypoints']) > 0: 
                        for idx, cl in enumerate(augmented['class_labels']):
                            if cl == 'L_eye': 
                                annotation['class'][0] = 1 
                                annotation['keypoints'][0] = augmented['keypoints'][idx][0]
                                annotation['keypoints'][1] = augmented['keypoints'][idx][1]
                            if cl == 'R_eye': 
                                annotation['class'][1] = 1 
                                annotation['keypoints'][2] = augmented['keypoints'][idx][0]
                                annotation['keypoints'][3] = augmented['keypoints'][idx][1]
                                
                annotation['keypoints'] = list(np.divide(annotation['keypoints'], [720,720,720,720]))


                with open(os.path.join(aug_path, partition, 'labels', f'{image.split(".")[0]}.{x}.json'), 'w') as f:
                    json.dump(annotation, f)

        except Exception as e:
            print(e)

In [7]:
aug_path = r'G:\Ayman\Future things\AI Projects\Iris Detection\iris_data\Iris Data Aug'

In [11]:
# Define the augmentation pipeline with additional augmentations
augmentor = alb.Compose([
    alb.HorizontalFlip(p=0.5),
    alb.VerticalFlip(p=0.5),
    alb.RandomBrightnessContrast(p=0.3),
    alb.RandomGamma(p=0.3),
    alb.RGBShift(p=0.3),
    alb.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, p=0.5),
    alb.GaussianBlur(p=0.3),
    alb.CLAHE(p=0.3)],
    keypoint_params=alb.KeypointParams(format='xy', label_fields=['class_labels'])
)

def process_image(image_path, json_path, save_img_dir, save_label_dir, augment=False, target_size=(500, 500)):
    with open(json_path, 'r') as f:
        data = json.load(f)

    # Check if 'shapes' key exists
    if 'shapes' not in data:
        print(f"Skipping {json_path} because it doesn't contain 'shapes'")
        return

    image = cv2.imread(image_path)
    keypoints = []
    class_labels = []
    
    for shape in data['shapes']:
        point = shape['points'][0]
        keypoints.append(point)
        class_labels.append(1)  # Assuming class labels are all 1, modify as needed

    if augment:
        augmented = augmentor(image=image, keypoints=keypoints, class_labels=class_labels)
        image = augmented['image']
        keypoints = augmented['keypoints']

    # Resize image to the target size
    original_h, original_w, _ = image.shape
    image = cv2.resize(image, target_size)

    # Adjust keypoints to new image size
    h, w = target_size
    adjusted_keypoints = [(kp[0] * w / original_w, kp[1] * h / original_h) for kp in keypoints]

    # Normalize keypoints
    normalized_keypoints = []
    for kp in adjusted_keypoints:
        normalized_keypoints.extend([kp[0] / w, kp[1] / h])

    # Generate new filenames for augmented data
    if augment:
        img_filename = f"{uuid.uuid1()}.jpg"
    else:
        img_filename = os.path.basename(image_path)

    save_img_path = os.path.join(save_img_dir, img_filename)

    # Save the image
    cv2.imwrite(save_img_path, image)

    # Create annotation in the required format
    new_annotation = {
        "image": img_filename,
        "class": class_labels,
        "keypoints": normalized_keypoints
    }

    # Save the new annotation
    save_json_path = os.path.join(save_label_dir, img_filename.replace('.jpg', '.json'))
    with open(save_json_path, 'w') as f:
        json.dump(new_annotation, f, indent=4)

def process_dataset(base_dir, augment_count=35, target_size=(500, 500)):
    for split in ['train', 'test', 'val']:
        image_dir = os.path.join(base_dir, split, 'images')
        label_dir = os.path.join(base_dir, split, 'labels')

        image_paths = glob(os.path.join(image_dir, '*.jpg'))

        for image_path in tqdm(image_paths):
            json_path = os.path.join(label_dir, os.path.basename(image_path).replace('.jpg', '.json'))

            # Process the original image and labels
            process_image(image_path, json_path, image_dir, label_dir, augment=False, target_size=target_size)
            
            # Apply augmentation and save results
            for _ in range(augment_count):  # Number of augmentations per image
                process_image(image_path, json_path, image_dir, label_dir, augment=True, target_size=target_size)


if __name__ == "__main__":
    process_dataset(aug_path, augment_count=35, target_size=(500, 500))

  3%|▎         | 4/116 [00:00<00:03, 30.89it/s]

Skipping G:\Ayman\Future things\AI Projects\Iris Detection\iris_data\Iris Data Aug\train\labels\26f54047-5bda-11ef-a436-c18859564890.json because it doesn't contain 'shapes'
Skipping G:\Ayman\Future things\AI Projects\Iris Detection\iris_data\Iris Data Aug\train\labels\26f54047-5bda-11ef-a436-c18859564890.json because it doesn't contain 'shapes'
Skipping G:\Ayman\Future things\AI Projects\Iris Detection\iris_data\Iris Data Aug\train\labels\26f54047-5bda-11ef-a436-c18859564890.json because it doesn't contain 'shapes'
Skipping G:\Ayman\Future things\AI Projects\Iris Detection\iris_data\Iris Data Aug\train\labels\26f54047-5bda-11ef-a436-c18859564890.json because it doesn't contain 'shapes'
Skipping G:\Ayman\Future things\AI Projects\Iris Detection\iris_data\Iris Data Aug\train\labels\26f54047-5bda-11ef-a436-c18859564890.json because it doesn't contain 'shapes'
Skipping G:\Ayman\Future things\AI Projects\Iris Detection\iris_data\Iris Data Aug\train\labels\26f54047-5bda-11ef-a436-c188595648

  9%|▉         | 11/116 [00:00<00:04, 25.61it/s]

Skipping G:\Ayman\Future things\AI Projects\Iris Detection\iris_data\Iris Data Aug\train\labels\2904729f-5bda-11ef-93ef-c18859564890.json because it doesn't contain 'shapes'
Skipping G:\Ayman\Future things\AI Projects\Iris Detection\iris_data\Iris Data Aug\train\labels\2904729f-5bda-11ef-93ef-c18859564890.json because it doesn't contain 'shapes'
Skipping G:\Ayman\Future things\AI Projects\Iris Detection\iris_data\Iris Data Aug\train\labels\2904729f-5bda-11ef-93ef-c18859564890.json because it doesn't contain 'shapes'
Skipping G:\Ayman\Future things\AI Projects\Iris Detection\iris_data\Iris Data Aug\train\labels\2904729f-5bda-11ef-93ef-c18859564890.json because it doesn't contain 'shapes'
Skipping G:\Ayman\Future things\AI Projects\Iris Detection\iris_data\Iris Data Aug\train\labels\2904729f-5bda-11ef-93ef-c18859564890.json because it doesn't contain 'shapes'
Skipping G:\Ayman\Future things\AI Projects\Iris Detection\iris_data\Iris Data Aug\train\labels\2904729f-5bda-11ef-93ef-c188595648

 12%|█▏        | 14/116 [00:00<00:03, 25.68it/s]

...

Skipping G:\Ayman\Future things\AI Projects\Iris Detection\iris_data\Iris Data Aug\val\labels\56945900-5bda-11ef-9a7f-c18859564890.json because it doesn't contain 'shapes'
Skipping G:\Ayman\Future things\AI Projects\Iris Detection\iris_data\Iris Data Aug\val\labels\56945900-5bda-11ef-9a7f-c18859564890.json because it doesn't contain 'shapes'
Skipping G:\Ayman\Future things\AI Projects\Iris Detection\iris_data\Iris Data Aug\val\labels\56945900-5bda-11ef-9a7f-c18859564890.json because it doesn't contain 'shapes'
Skipping G:\Ayman\Future things\AI Projects\Iris Detection\iris_data\Iris Data Aug\val\labels\56945900-5bda-11ef-9a7f-c18859564890.json because it doesn't contain 'shapes'
Skipping G:\Ayman\Future things\AI Projects\Iris Detection\iris_data\Iris Data Aug\val\labels\56945900-5bda-11ef-9a7f-c18859564890.json because it doesn't contain 'shapes'
Skipping G:\Ayman\Future things\AI Projects\Iris Detection\iris_data\Iris Data Aug\val\labels\56945900-5bda-11ef-9a7f-c18859564890.json beca

 75%|███████▌  | 12/16 [00:00<00:00, 20.70it/s]

Skipping G:\Ayman\Future things\AI Projects\Iris Detection\iris_data\Iris Data Aug\val\labels\574be3c3-5bda-11ef-8833-c18859564890.json because it doesn't contain 'shapes'
Skipping G:\Ayman\Future things\AI Projects\Iris Detection\iris_data\Iris Data Aug\val\labels\574be3c3-5bda-11ef-8833-c18859564890.json because it doesn't contain 'shapes'
Skipping G:\Ayman\Future things\AI Projects\Iris Detection\iris_data\Iris Data Aug\val\labels\574be3c3-5bda-11ef-8833-c18859564890.json because it doesn't contain 'shapes'
Skipping G:\Ayman\Future things\AI Projects\Iris Detection\iris_data\Iris Data Aug\val\labels\574be3c3-5bda-11ef-8833-c18859564890.json because it doesn't contain 'shapes'
Skipping G:\Ayman\Future things\AI Projects\Iris Detection\iris_data\Iris Data Aug\val\labels\574be3c3-5bda-11ef-8833-c18859564890.json because it doesn't contain 'shapes'
Skipping G:\Ayman\Future things\AI Projects\Iris Detection\iris_data\Iris Data Aug\val\labels\574be3c3-5bda-11ef-8833-c18859564890.json beca

100%|██████████| 16/16 [00:00<00:00, 21.24it/s]

Skipping G:\Ayman\Future things\AI Projects\Iris Detection\iris_data\Iris Data Aug\val\labels\591c22b9-5bda-11ef-92b1-c18859564890.json because it doesn't contain 'shapes'
Skipping G:\Ayman\Future things\AI Projects\Iris Detection\iris_data\Iris Data Aug\val\labels\591c22b9-5bda-11ef-92b1-c18859564890.json because it doesn't contain 'shapes'
Skipping G:\Ayman\Future things\AI Projects\Iris Detection\iris_data\Iris Data Aug\val\labels\591c22b9-5bda-11ef-92b1-c18859564890.json because it doesn't contain 'shapes'
Skipping G:\Ayman\Future things\AI Projects\Iris Detection\iris_data\Iris Data Aug\val\labels\591c22b9-5bda-11ef-92b1-c18859564890.json because it doesn't contain 'shapes'
Skipping G:\Ayman\Future things\AI Projects\Iris Detection\iris_data\Iris Data Aug\val\labels\591c22b9-5bda-11ef-92b1-c18859564890.json because it doesn't contain 'shapes'
Skipping G:\Ayman\Future things\AI Projects\Iris Detection\iris_data\Iris Data Aug\val\labels\591c22b9-5bda-11ef-92b1-c18859564890.json beca

In [45]:
def convert_json_format(original_json_path, image_width, image_height):
    try:
        # Load the original JSON data
        with open(original_json_path, 'r') as f:
            data = json.load(f)
        
        # Check if 'imagePath' exists in the JSON
        if 'imagePath' in data:
            image_name = data['imagePath']
        else:
            # Handle cases where 'imagePath' is missing
            print(f"Key 'imagePath' not found in {original_json_path}. Skipping file.")
            return
        
        # Initialize class list and keypoints list
        class_list = []
        keypoints = []
        
        # Extract keypoints and normalize them
        for shape in data['shapes']:
            point = shape['points'][0]
            normalized_x = point[0] / image_width
            normalized_y = point[1] / image_height
            keypoints.extend([normalized_x, normalized_y])
            class_list.append(1)  # Assuming all keypoints have class 1, adjust if needed
        
        # Create the new JSON structure
        new_format = {
            "image": image_name,
            "class": class_list,
            "keypoints": keypoints
        }
        
        # Overwrite the original JSON file with the new format
        with open(original_json_path, 'w') as f:
            json.dump(new_format, f, indent=4)

    except Exception as e:
        print(f"An error occurred while processing {original_json_path}: {e}")

def process_directory(labels_dir, image_width, image_height):
    # Process all JSON files in the labels directory
    for filename in os.listdir(labels_dir):
        if filename.endswith('.json'):
            file_path = os.path.join(labels_dir, filename)
            convert_json_format(file_path, image_width, image_height)
            print(f"Processed and overwritten: {filename}")

# Example usage:
labels_dir = r'G:\Ayman\Future things\AI Projects\Iris Detection\iris_data\aug_data\val\labels'  # Path to your labels directory
image_width = 500  # Replace with actual image width
image_height = 500  # Replace with actual image height

# Process all JSON files in the directory
process_directory(labels_dir, image_width, image_height)

Processed and overwritten: 53ff0eed-5bda-11ef-94af-c18859564890.json
Processed and overwritten: 545c883d-5bda-11ef-9895-c18859564890.json
Processed and overwritten: 54bbea88-5bda-11ef-a93d-c18859564890.json
Processed and overwritten: 551826a3-5bda-11ef-a970-c18859564890.json
Processed and overwritten: 557776ca-5bda-11ef-97cc-c18859564890.json
Processed and overwritten: 55d8b504-5bda-11ef-9fb4-c18859564890.json
Processed and overwritten: 5634fe43-5bda-11ef-8adc-c18859564890.json
Processed and overwritten: 56945900-5bda-11ef-9a7f-c18859564890.json
Processed and overwritten: 56ef70f7-5bda-11ef-bee1-c18859564890.json
Processed and overwritten: 574be3c3-5bda-11ef-8833-c18859564890.json
Processed and overwritten: 57a8f03b-5bda-11ef-a882-c18859564890.json
Processed and overwritten: 5808023f-5bda-11ef-9ac0-c18859564890.json
Processed and overwritten: 58bf6a45-5bda-11ef-948d-c18859564890.json
Processed and overwritten: 591c22b9-5bda-11ef-92b1-c18859564890.json
Processed and overwritten: 597bbd9